# NVL Python Wrapper

NVL (Neo4j Visualization Library) is a JavaScript library developed to display large graphs. This notebook showcases an experimental Python wrapper for this libary.

Below is a basic connection to a Neo4j database. We use the `Result.graph` result transformer to map the result to a graph object and store the result in `result`.

In [ ]:
%pip install neo4j
%pip install nvl_package

In [ ]:
import os

from neo4j import GraphDatabase, Result

URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")

auth = None
if os.environ.get("NEO4J_USER") and os.environ.get("NEO4J_PASSWORD"):
    auth = (os.environ.get("NEO4J_USER"), os.environ.get("NEO4J_PASSWORD"))

with GraphDatabase.driver(URI, auth=auth) as driver:
    driver.verify_connectivity()

    result = driver.execute_query(
        "MATCH (n)-[r]-(m) RETURN n,r,m LIMIT 10", database_="neo4j", result_transformer_=Result.graph
    )

    result

result

Below we map the graph object's nodes and relationships to the correct format for NVL. The Node and Relationship type documentation can be found in NVL's docs: https://neo4j.com/docs/nvl/current/core-library/#_nodes

In [ ]:
def map_nodes(n):
    caption = ""
    for label in n.labels:
        caption += label
    return {"id": str(n.element_id), "caption": caption}


def map_relationships(r):
    return {
        "id": str(r.element_id),
        "from": str(r.start_node.element_id),
        "to": str(r.end_node.element_id),
        "caption": str(r.type),
    }


nodes = list(map(map_nodes, result.nodes))
relationships = list(map(map_relationships, result.relationships))

nodes

Now, we can render our result with NVL the following way:

In [ ]:
from neo4j_viz.nvl import NVL

my_nvl = NVL()

my_nvl

In [ ]:
my_nvl.render(nodes=nodes, relationships=relationships)

There are two renderers available in NVL. A WebGL renderer used for displaying large graphs with high performance and a Canvas renderer, which renders the graph with lower performance but in more detail with captions and arrowheads. The WebGL renderer is used by default. To use the Canvas renderer, you can enable it by passing `{ "renderer": "webgl" }` as a third `option` parameter to NVL.

In [ ]:
my_nvl.render(nodes=nodes, relationships=relationships, options={"renderer": "webgl"})